In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size':15})

In [2]:
df = pd.read_csv("new1_df.csv",encoding='cp932')

In [3]:
# df['受付日']=pd.to_datetime(df['受付日'],format='%Y%m%d')
df.index = df['受付日']
df.sort_index(inplace=True)
del df['受付日']
df = df["2016-04-01":"2017-04-30"]

In [5]:
df1 = df.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396540 entries, 2016-04-01 to 2017-03-31
Data columns (total 6 columns):
受付NO       396540 non-null object
部品品番       396540 non-null object
大現象名       347273 non-null object
修理依頼内容     396540 non-null object
一発修理FLG    396540 non-null int64
商品品番       396521 non-null object
dtypes: int64(1), object(5)
memory usage: 21.2+ MB


In [11]:
df.head()

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,L2160138,P20-126,漏水,シャワーを使うと向かって右から水漏れ,0,CW-RY100
2016-04-01,L2160133,371-1004A,温風(ﾄﾞﾗｲ)関係,温風乾燥の温風が出ない。,0,DWT-MM55
2016-04-01,L1956664,343-1001D,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
2016-04-01,L1956664,50-156,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
2016-04-01,L1956666,50-2318,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,□おまかせ点検,1,DV-115A


# メモリ削減のため、受付NOと部品品番をラベルエンコーディングする

In [4]:
import category_encoders as ce

In [5]:
# Eoncodeしたい列をリストで指定。もちろん複数指定可能。
list_cols = ['受付NO']

# OneHotEncodeしたい列を指定。Nullや不明の場合の補完方法も指定。
ce_ohe = ce.OrdinalEncoder(cols=list_cols,handle_unknown='impute')

# pd.DataFrameをそのまま突っ込む
df = ce_ohe.fit_transform(df)


In [8]:
# Eoncodeしたい列をリストで指定。もちろん複数指定可能。
parts_cols = ['部品品番']

# OneHotEncodeしたい列を指定。Nullや不明の場合の補完方法も指定。
pa_ohe = ce.OrdinalEncoder(cols=parts_cols,handle_unknown='impute')

# pd.DataFrameをそのまま突っ込む
df = pa_ohe.fit_transform(df)


In [9]:
df.head()

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,1,1,漏水,シャワーを使うと向かって右から水漏れ,0,CW-RY100
2016-04-01,2,2,温風(ﾄﾞﾗｲ)関係,温風乾燥の温風が出ない。,0,DWT-MM55
2016-04-01,3,3,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
2016-04-01,3,4,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
2016-04-01,4,5,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,□おまかせ点検,1,DV-115A


In [10]:
parts_df = pd.concat([df1['部品品番'].rename('parts_name'), df['部品品番'].rename('parts_label')], axis=1)

In [11]:
parts_df.head()

,parts_name,parts_label
受付日,,
2016-04-01,P20-126,1
2016-04-01,371-1004A,2
2016-04-01,343-1001D,3
2016-04-01,50-156,4
2016-04-01,50-2318,5


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422642 entries, 2016-04-01 to 2017-04-30
Data columns (total 6 columns):
受付NO       422642 non-null int32
部品品番       422642 non-null int32
大現象名       369449 non-null object
修理依頼内容     422642 non-null object
一発修理FLG    422642 non-null int64
商品品番       422623 non-null object
dtypes: int32(2), int64(1), object(3)
memory usage: 19.3+ MB


# 欠損値の処理

In [13]:
df.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
False,422642,422642,369449,422642,422642,422623
True,0,0,53193,0,0,19


### 欠損値があるデータを見る

In [41]:
df[df.isnull().any(axis=1)]

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,L1956647,GBC-900S,NaN,トイレにカギを落としてしまい詰まりがひどくなってきた,1,GBC-900S
2016-04-01,L1956647,351-1038A,NaN,トイレにカギを落としてしまい詰まりがひどくなってきた,1,GBC-900S
2016-04-01,L1956501,CWA-29,NaN,おまかせ点検,1,CW-K111
2016-04-01,L1956501,CW-K111,NaN,おまかせ点検,1,CW-K111
2016-04-01,L1956459,CW-E41,NaN,相談（１階）,1,CW-E41
2016-04-01,L1956443,CW-K111,NaN,おまかせ点検,1,CW-K111
2016-04-01,L1956459,E53-100,NaN,相談（１階）,1,CW-E41
2016-04-01,L1956821,34-1234,NaN,２Ｆ/オーバーフロー管折れ,1,ﾛ-ﾀﾝｸ
2016-04-01,L1956821,50-1001,NaN,２Ｆ/オーバーフロー管折れ,1,ﾛ-ﾀﾝｸ


### おまかせ点検　を含むデータを確認,削除

In [42]:
df[df.修理依頼内容.str.contains('おまかせ点検')]

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,L1956666,50-2318,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,□おまかせ点検,1,DV-115A
2016-04-01,L1956666,343-1008D,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,□おまかせ点検,1,DV-115A
2016-04-01,L1956666,331-1012D,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,□おまかせ点検,1,DV-115A
2016-04-01,L1956501,CWA-29,NaN,おまかせ点検,1,CW-K111
2016-04-01,L1956501,CW-K111,NaN,おまかせ点検,1,CW-K111
2016-04-01,L1956443,CW-K111,NaN,おまかせ点検,1,CW-K111
2016-04-01,L2238212,CW-E43,NaN,おまかせ点検,1,CW-E43
2016-04-01,L2238211,K100-300,便座･便蓋関係,おまかせ点検,1,CW-E43
2016-04-01,L2238207,CW-K33,NaN,おまかせ点検,1,CW-K33


In [15]:
len(df)

396540

In [43]:
len(df[df.修理依頼内容.str.contains('おまかせ点検')])

27313

In [44]:
len(df[~df.修理依頼内容.str.contains('おまかせ点検')])

369227

In [6]:
df = df[~df.修理依頼内容.str.contains('おまかせ点検')]

### 相談　を含むデータを確認,削除

In [46]:
df[df.修理依頼内容.str.contains('相談')]

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,L1956459,CW-E41,NaN,相談（１階）,1,CW-E41
2016-04-01,L1956459,E53-100,NaN,相談（１階）,1,CW-E41
2016-05-18,L2255883,TK-KA-100,止水不良(便鉢内漏水),メンテ田中氏に相談済,1,CW-KB21
2016-05-23,L2394310,C-89,NaN,トイレの取替え相談希望,1,C-89SF
2016-06-10,L1946079,DT-3512X,ｼｬﾜｰ･ﾋﾞﾃﾞ関係,代替え相談,1,DT-3512X
2016-06-11,L2174232,290-1044C,NaN,浜松影山課長へ相談分,1,DTV283
2016-06-11,L2174232,61-1134,NaN,浜松影山課長へ相談分,1,DTV283
2016-06-11,L2395220,GBC-ZA10,漏水,買替え相談,1,DT-511
2016-06-24,L1946546,DWT-MC83,その他の不具合,取り替え相談,1,DWT-MC83


In [47]:
len(df[df.修理依頼内容.str.contains('相談')])

35

In [48]:
len(df)

369227

In [49]:
len(df[~df.修理依頼内容.str.contains('相談')])

369192

In [7]:
df = df[~df.修理依頼内容.str.contains('相談')]

### 製品在庫処理　を含むデータを確認,削除

In [51]:
df[df.修理依頼内容.str.contains("製品在庫処理")]

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2017-03-31,L1955067,GL-2295,NaN,製品在庫処理,1,-
2017-03-31,L1955067,UDY-8002006UR29S1-LH-S,NaN,製品在庫処理,1,-
2017-03-31,L1955067,DWT-MC53-IMT,NaN,製品在庫処理,1,-
2017-03-31,L1955067,CW-KB23,NaN,製品在庫処理,1,-
2017-03-31,L1955067,GBC-S11S,NaN,製品在庫処理,1,-
2017-03-31,L1955067,DV-S516,NaN,製品在庫処理,1,-


In [8]:
df = df[~df.修理依頼内容.str.contains('製品在庫処理')]

### 販売、取付　を含むデータを確認,削除

In [55]:
df[df.修理依頼内容.str.contains("販売")]

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-08,L2240533,CW-J120,止水不良(便鉢内漏水),販売・取付,1,CW-J120
2016-04-18,L1965038,CW-KB21,NaN,販売・取付,1,DT-810
2016-04-19,L1944541,TF-20B,止水不良(便鉢内漏水),シャワートイレ販売取り付け,1,DT-680S
2016-04-19,L1944541,TF-10R-L,止水不良(便鉢内漏水),シャワートイレ販売取り付け,1,DT-680S
2016-04-21,L1966914,DT-B153,便座･便蓋関係,販売・取付,0,DT-B153
2016-04-21,L1966914,311-1034,便座･便蓋関係,販売・取付,0,DT-B153
2016-05-16,L1978408,TF-10R-L,止水不良(便鉢内漏水),【らいさぽ】ｼｬﾜ-ﾄｲﾚ販売取付,1,DT-2820
2016-06-04,L2394888,A-3334,外観関係(ｶﾞﾀﾂｷ含む),水栓金具販売取付,1,DT-4820
2016-06-09,L1991610,DT-680S,止水不良(便鉢内漏水),販売取り付け希望/1F,1,DT-680S


In [56]:
df[df.修理依頼内容.str.contains("取付")]

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,L2160168,CW-H41,NaN,ｼｬﾜ-ﾄｲﾚの取付。,1,CW-H41
2016-04-01,L2160207,K30-126A,漏水,給水ホース取付部より漏水,1,CW-K33
2016-04-01,L1956489,TF-Z180A,外観関係(ｶﾞﾀﾂｷ含む),男子ﾄｲﾚ:ﾚﾊﾞ-取付部が折れて、水が流せない。,0,DT-Z150U
2016-04-02,L2160366,DV-S516,異音関係,シャワーホースの取付希望,1,DV-S516
2016-04-02,L1957126,43-3010,外観関係(ｶﾞﾀﾂｷ含む),便座取付部の破損。修理依頼。,1,CW-EA14Q
2016-04-02,L1956928,YBC-S20S,NaN,展示場ｻﾃｨｽを取外し事務所へ取付（既存品は撤去処分して下さい）,1,DV-S626-R
2016-04-02,L1956928,DV-S626-R,NaN,展示場ｻﾃｨｽを取外し事務所へ取付（既存品は撤去処分して下さい）,1,DV-S626-R
2016-04-04,L2239270,CW-B51,NaN,シャワートイレ取付,1,CW-B51-LP
2016-04-04,L1957501,213-1020,外観関係(ｶﾞﾀﾂｷ含む),□取付時から便座がガタツク□向左付け根便座から便器内へ水漏れ,1,CW-EA11QC


In [57]:
df.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
False,369186,369186,342134,369186,369186,369167
True,0,0,27052,0,0,19


In [59]:
df[df.isnull().any(axis=1)].drop_duplicates(subset = "修理依頼内容")

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付日,,,,,,
2016-04-01,L1956647,GBC-900S,NaN,トイレにカギを落としてしまい詰まりがひどくなってきた,1,GBC-900S
2016-04-01,L1956821,34-1234,NaN,２Ｆ/オーバーフロー管折れ,1,ﾛ-ﾀﾝｸ
2016-04-01,L2392172,311-1096,NaN,□便座の熱線部分が黄色に変色□便座が倒れる,1,DV-S418
2016-04-01,L2238086,322-1034,NaN,水抜き栓の所から水漏れ,1,DT-5823U
2016-04-01,L2238255,DV-118A,NaN,本体に電気が入っていない。,1,DV-118A
2016-04-01,L2238168,50-1001-2,NaN,タンクより水が漏れているとの事。パッキンの交換？,1,T-527
2016-04-01,L2238116,DT-C182U,NaN,□シャワーが噴射しない（１階）,1,DT-C182U
2016-04-01,L2392150,BC-120SU,NaN,1F/ﾄｲﾚ詰まり。異物は落としていない。,1,BC-120SU
2016-04-01,L2392151,GBC-320S,NaN,ﾄｲﾚ詰り,1,GBC-320S


### 欠損値を含むデータを削除

In [9]:
df.dropna(inplace=True)

In [10]:
df.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,受付NO,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
False,342115,342115,342115,342115,342115,342115


### データの形を変える（カラムに部品名）

In [18]:
df.shape

(364008, 6)

In [13]:
len(df.部品品番.unique())

7004

In [10]:
df = df.set_index('受付NO')

In [20]:
df.head(20)

,部品品番,大現象名,修理依頼内容,一発修理FLG,商品品番
受付NO,,,,,
1,1,漏水,シャワーを使うと向かって右から水漏れ,0,CW-RY100
2,2,温風(ﾄﾞﾗｲ)関係,温風乾燥の温風が出ない。,0,DWT-MM55
3,3,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
3,4,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
5,8,止水不良(便鉢内漏水),タンク内で水漏れの音がして便器内にチョロチョロと水漏れ,1,DT-4573
5,9,止水不良(便鉢内漏水),タンク内で水漏れの音がして便器内にチョロチョロと水漏れ,1,DT-4573
3,10,漏水,ﾉｽﾞﾙ水もれ戻らない、ｼｬﾜ-使用時ﾀﾝｸ下から水もれ,1,DT-3813
6,9,止水不良(便鉢内漏水),水が止まらない。ﾀﾝｸから便器へ水が流れ続ける。,0,DT-3512XW
6,11,止水不良(便鉢内漏水),水が止まらない。ﾀﾝｸから便器へ水が流れ続ける。,0,DT-3512XW


In [11]:
outcome = pd.get_dummies(df['部品品番'])

In [12]:
outcome.head()

,)CW-KA21,04700G-1,09840P,1,10-152,100-140,1000,1000-100,1000-122A,1000-124,...,YHBC-CH10S,YHBC-G20S,YHBC-ZA10H,YL-82-SET,YL-A74HC,YL-DA82SKH12B,YN-AALEABKX,YT-891,Z1T,ZA180
受付NO,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
outcome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364008 entries, 1 to 256217
Columns: 7184 entries, )CW-KA21 to ZA180
dtypes: uint8(7184)
memory usage: 2.4 GB


In [ ]:
outcome = outcome.groupby(outcome.index).agg(sum)

In [15]:
# outcome.groupby(outcome.index).apply(lambda x: x.sum())

KeyboardInterrupt: 